Question5. Write a program that can encrypt and Decrypt using a 2 X 2 Hill Cipher.

In [2]:
#Python program to illustrate 2 X 2 Hill Cipher Technique.

import numpy as np

# Defining function for conversion of charater to an integer. 
def chr_to_int(char):
    # Uppercase the char to get into range 65-90 in ascii table.
    char = char.upper()
    # Casting chr to int and subtract 65 to get 0-25.
    integer = ord(char) - 65
    return integer

# Function to create matrix corresponding to key text.
def make_key_matrix(keyText):
    # Defining 2 x 2 key matrix.
    key_matrix =   np.zeros((2, 2), dtype=np.int32)
    key_matrix[0][0] = chr_to_int(keyText[0])
    key_matrix[0][1] = chr_to_int(keyText[1])
    key_matrix[1][0] = chr_to_int(keyText[2])
    key_matrix[1][1] = chr_to_int(keyText[3])
    
    # Returning key matrix.    
    return key_matrix
    

# Function for creating integers matrix corresponding to characters in message text.
def create_matrix_of_integers_from_text(string):
    # Maping characters in string to a list of integers i.e. a/A <-> 0, b/B <-> 1 ... z/Z <-> 25.
    integers = [chr_to_int(c) for c in string]
    length = len(integers)
    integer_matrix = np.zeros((2, int(length / 2)), dtype=np.int32)
    iterator = 0
    for column in range(int(length / 2)):
        for row in range(2):
            integer_matrix[row][column] = integers[iterator]
            iterator += 1
    return integer_matrix
    
    
def encrypt(plainText,key):
    
    """
        Purpose : Function to perform Encryption using 2x2 Hill Cipher.
        Input   : plainText = string value of plain text to be encoded using 2 x 2 Hill Cipher.
                  key = string value of keyword use to encrypt plain text.  
        Output  : Returns cipher text encrypted using 2 x 2 Hill Cipher. 
    """
    
    # Create Key matrix for Encryption.
    key_matrix = make_key_matrix(key)
    
    """
    If determinant of key matrix is 0 meaning matrix is not invertible or if matrix 
    contains values greater than 26 or less than 0, so we cannot encrypt plain text.
    """
    # Calculating determinant of key matrix.
    determinant = key_matrix[0][0]*key_matrix[1][1]-key_matrix[0][1]*key_matrix[1][0]
    multiplicative_inverse = find_multiplicative_inverse(determinant)
    if determinant == 0:
        print("\nDeterminant is 0, so key matrix is not invertible.\nCannot Encrypt.")
        return -1
    elif np.amax(key_matrix) > 26 and np.amin(key_matrix) < 0:
        print("\nOnly a-z characters are accepted.\nCannot Encrypt.")
        print(np.amax(key_matrix), np.amin(key_matrix))
        return -1
    elif multiplicative_inverse == -1:
            print("\nDeterminant is not relatively prime to 26, not invertible key.\nCannot Encrypt.")
            return -1
    else:
        # Replace spaces with nothing in plainText.
        plainText = plainText.replace(" ", "")

        # Append zero if the plainText is not divisble by 2.
        len_check = len(plainText) % 2 == 0
        if not len_check:
            plainText += "0"

        # Create integers matrix for plain text.
        plainText_Matrix = create_matrix_of_integers_from_text(plainText)

        # Calculate length of the plainText
        text_len = int(len(plainText) / 2)

        # Calculating cipherText = key_matrix * plainText_Matrix i.e. c = Kp (mod 26)
        cipherText = ""
        for i in range(text_len):
            # Calculating product for first row.
            row_0 = plainText_Matrix[0][i] * key_matrix[0][0] + plainText_Matrix[1][i] * key_matrix[0][1]
            # Reducing to modulo 26 and add 65 to get back to the A-Z range in ascii.
            integer = int(row_0 % 26 + 65)
            # Converting back to letters.
            cipherText += chr(integer)
            # Repeat above three steps for the second row.
            row_1 = plainText_Matrix[0][i] * key_matrix[1][0] + plainText_Matrix[1][i] * key_matrix[1][1]
            integer = int(row_1 % 26 + 65)
            cipherText += chr(integer)

        #Return the encrypted text.
        return cipherText


# Function for calculating multiplicative inverse of determinant.
def find_multiplicative_inverse(determinant):
    multiplicative_inverse = -1
    for i in range(26):
        inverse = determinant * i
        if inverse % 26 == 1:
            multiplicative_inverse = i
            break
    return multiplicative_inverse


def decrypt(cipherText,key):
    """
        Purpose : Function to perform Decryption using 2x2 Hill Cipher.
        Input   : cipherText = string value of cipher text to be decoded using 2 x 2 Hill Cipher.
                  key = string value of keyword use to decrypt cipher text.  
        Output  : Returns plain text decrypted using 2 x 2 Hill Cipher. 
    """
    # Create Key matrix for Decryption.
    key_matrix = make_key_matrix(key)    
    
    """
    If determinant of key matrix is 0 meaning matrix is not invertible or if matrix 
    contains values greater than 26 or less than 0, so we cannot encrypt plain text.
    """
    # Calculating determinant of key matrix.
    determinant = key_matrix[0][0]*key_matrix[1][1]-key_matrix[0][1]*key_matrix[1][0]
    multiplicative_inverse = find_multiplicative_inverse(determinant)
    
    if determinant == 0:
        print("\nDeterminant is 0, so key matrix is not invertible.\nCannot Decrypt.")
        return -1
    elif np.amax(key_matrix) > 26 and np.amin(key_matrix) < 0:
        print("\nOnly a-z characters are accepted.\nCannot Decrypt.")
        print(np.amax(key_matrix), np.amin(key_matrix))
        return -1
    elif multiplicative_inverse == -1:
            print("\nDeterminant is not relatively prime to 26, not invertible key.\nCannot Decrypt.")
            return -1

    determinant = determinant % 26   
    
    inverse_key_matrix = key_matrix
    # Swapping a <-> d for 2 x 2 matrix Inverse.
    inverse_key_matrix[0][0], inverse_key_matrix[1][1] = inverse_key_matrix[1, 1], inverse_key_matrix[0, 0]
    # Replacing for 2 x 2 matrix Inverse.
    inverse_key_matrix[0][1] *= -1
    inverse_key_matrix[1][0] *= -1
    # Calucalting Inverse matrix.
    for row in range(2):
        for column in range(2):
            inverse_key_matrix[row][column] *= multiplicative_inverse
            inverse_key_matrix[row][column] = inverse_key_matrix[row][column] % 26
    
    # Create integers matrix for cipher text.
    cipherText_matrix = create_matrix_of_integers_from_text(cipherText)
    msg_len = int(len(cipherText) / 2)
    plainText = ""
    for i in range(msg_len):
        # Calculating product for first row.
        column_0 = cipherText_matrix[0][i] * inverse_key_matrix[0][0] + cipherText_matrix[1][i] * inverse_key_matrix[0][1]
        # Reducing to modulo 26 and add 65 to get back to the A-Z range in ascii.
        integer = int(column_0 % 26 + 65)
        # Converting back to letters.
        plainText += chr(integer)
        # Repeat above three steps for the second row.
        column_1 = cipherText_matrix[0][i] * inverse_key_matrix[1][0] + cipherText_matrix[1][i] * inverse_key_matrix[1][1]
        integer = int(column_1 % 26 + 65)
        plainText += chr(integer)
    if plainText[-1]=='0':
        plainText=plainText[:-1]
    #Return the decrypted text.
    return plainText


# Defining Main Function.
def main():
    print("Hill Cipher ...")
    plainText = input("\nEnter the plain text you want to encrypt : ")
    key = input("\nEnter 4 letter keyword : ")
    cipherText = encrypt(plainText,key)
    if(cipherText!=-1):
        print("\nEncrypted message (cipher text) is : ",cipherText)
    plainText = decrypt(cipherText,key)
    if(plainText!=-1):
        print("\nDecrypted message (plain text) is : ",plainText)    
    
# Calling main function
if __name__ == "__main__":
    main()

Hill Cipher ...

Enter the plain text you want to encrypt : short terms

Enter 4 letter keyword : hill

Encrypted message (cipher text) is :  APADZCIXUS

Decrypted message (plain text) is :  SHORTTERMS
